In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns

iris = load_iris()
X = iris.data       
y = iris.target     

df = pd.DataFrame(X, columns=iris.feature_names)

In [ ]:
# X.shape
X.shape[0]

(150, 4)

1.  On place k  (centroïdes) au hasard parmi les données

In [39]:
# Nombre de centroïde
k = 4
#  k indices différents et aléatoires parmi les lignes de X, sans répétition.
indices = np.random.choice(X.shape[0], k, replace=False)
# tableau (k, 4) contenant k centroïdes initiaux choisis parmi les points du dataset.
centroids = X[indices]

centroids.shape

(4, 4)

2.  Chaque point du dataset est affecté au centroïd le plus proche


- `np.newaxis` →      ajoute une dimension

- `X[:, np.newaxis]` →  transforme X de (150, 4) en (150, 1, 4)

- `centroids.shape` →  (k, 4)

- `X[:, np.newaxis - centroids]` →  (150, k, 4)

- `np.linalg.norm(..., axis=2)` →  calcule la distance euclidienne → forme finale : (150, k)

- `np.argmin(..., axis=1)` →  donne pour chaque ligne, l’indice du centroïde le plus proche

In [ ]:
def assign_to_clusters(X, centroids):
    # Calcul des distances entre chaque point et chaque centroïde
    distances = np.linalg.norm(X[:, np.newaxis] - centroids, axis=2)
    
    # Affectation : indice du centroïde le plus proche
    cluster_labels = np.argmin(distances, axis=1)
    
    return cluster_labels    

In [42]:
assign_to_clusters(X, centroids)


array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 1, 1, 1, 3, 1, 1, 1, 3, 1, 3, 3, 3, 3, 1, 3, 1,
       1, 3, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 1, 1, 1, 1, 1,
       3, 3, 3, 1, 3, 3, 3, 3, 3, 1, 3, 3, 0, 1, 0, 0, 0, 2, 3, 2, 0, 2,
       0, 0, 0, 1, 1, 0, 0, 2, 2, 1, 0, 1, 2, 1, 0, 2, 1, 1, 0, 0, 2, 2,
       0, 1, 1, 2, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1], dtype=int64)

3.   Le centroïd est déplacé au milieu du 
   cluster (la moyenne des points)

- `labels == i` sélectionne tous les points du cluster 𝑖

- `.mean(axis=0)` calcule la moyenne colonne par colonne

- `np.zeros((k, 4))` initialise la nouvelle matrice de centroïdes

In [ ]:
def update_centroids(X, labels, k):
    new_centroids = np.zeros((k, X.shape[1]))
    for i in range(k):
        cluster_points = X[labels == i]
        if len(cluster_points) > 0:
            new_centroids[i] = cluster_points.mean(axis=0)
    return new_centroids


4.  on recommence jusqu'à ce que les centroïds 
    convergent vers le point d'équilibre